<a href="https://colab.research.google.com/github/cimbelli/Surface-Heat-Islands/blob/main/temperatura_max_estiva_capoluoghi-regione_1.14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stima delle temperatura massima mensile nei capoluoghi di regione

In [1]:
import os
import ee
import pandas as pd
import json
import geopandas as gpd
import requests


ee.Authenticate()
ee.Initialize()
!pip install rasterstats
!pip install wget
import wget
from rasterstats import zonal_stats

# scegli quale statistica estrarre per sezione sulla base delle temperature massime estratte per pixel
statistiche = "min mean max median"
stat_map = "median"

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bnh6WXu7aYH_CMfhlRU4B-NfhGKOO7yrN8KbVzR5l28&tc=mYqblXbaC-UKdlfZtVmQngWltwdPWZZF5vcWh0IPitM&cc=viPNEkPWnA_2IY-w54BtIXqP7VtkiyeMuAhAhcD--Zw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXltKhOpLOpP3TXXhjbh2qM9qa7Xf_pEFqotQwMGjOtS8-FaZ_vtScQ

Successfully saved authorization token.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl

In [8]:
anni = [2019, 2020, 2021, 2022, 2023]
#mesi = [6,7,8,9]

In [3]:
import folium
from folium import plugins

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [4]:
def scarica(lyr, name, aoi):
    url = lyr.getDownloadUrl({
        'name': name[:-4],
        'bands': ['ST'],
        'region': aoi,
        'scale': 30,
        'format': 'GEO_TIFF'
    })
    response = requests.get(url)
    with open(name, 'wb') as fd:
        fd.write(response.content)
    return None

In [5]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [6]:
# carica lo shape dei capoluoghi di regione
nomecap = 'caporeg.geojson'
if not(os.path.isfile(nomecap)):
  #wget.download('https://github.com/cimbelli/Surface-Heat-Islands/raw/5006ef2e37091eedf421f65ed9cbf80c15ccbdfd/' + nomecap)
  wget.download('https://raw.githubusercontent.com/cimbelli/Surface-Heat-Islands/main/' + nomecap)
g = open(nomecap)
cc = json.load(g)

gdf = gpd.read_file(nomecap)
centro = gdf.to_crs(epsg=3857).centroid.to_crs(gdf.crs)
#gdf["lon"] = gdf.to_crs(epsg=3857).centroid.to_crs(gdf.crs) #gdf["geometry"].to_crs(4326).centroid.x
#gdf["lat"] = gdf["geometry"].centroid.y
gdf["lon"] = centro.x
gdf["lat"] = centro.y

In [ ]:
for anno in anni:
    for i in range(len(cc['features'])):           # for all the records
    #for i in range(17,len(cc['features'])):

        try:
          del sdf, df_o
        except:
          pass

        reg = cc['features'][i]['properties']['COD_REG']
        reg1 = ('0' if reg < 10 else '') + str(reg)
        id1 = cc['features'][i]['properties']['PRO_COM']
        procom1 = '0' * (6-len(str(id1))) + str(id1)

        coords = cc['features'][i]['geometry']['coordinates']
        fc = ee.FeatureCollection(cc['features'])
        comune = cc['features'][i]['properties']['COMUNE']
        print('---',comune ,'---')


        nomefilesez = 'sez11_' + procom1 +'.geojson'
        if not(os.path.isfile(nomefilesez)):
          urlfilesez = 'https://raw.githubusercontent.com/cimbelli/Surface-Heat-Islands/main/sez11/' + nomefilesez
          wget.download(urlfilesez)
        g1 = open(nomefilesez)
        df_sez11 = gpd.read_file(g1)
        df_sez11['PRO_COM'] = df_sez11['PRO_COM'].astype(int)
        df_sez11['SEZ'] = df_sez11['SEZ'].astype(int)

        lat = gdf[gdf['PRO_COM']== id1]['lat'].values[0]
        lon = gdf[gdf['PRO_COM']== id1]['lon'].values[0]

        minx = gdf[gdf['PRO_COM']== id1].geometry.bounds.minx.values[0]
        maxx = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxx.values[0]
        miny = gdf[gdf['PRO_COM']== id1].geometry.bounds.miny.values[0]
        maxy = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxy.values[0]

        region = ee.Geometry.BBox(minx, miny, maxx, maxy)

        #for mese in mesi:
        nome_img = 'ST_' + comune + '_'+ str(anno) + '.tif'
        #mese2 = ('0' + str(mese+1)) if mese < 9 else str(mese+1)

        if not(os.path.isfile(nome_img)):

            L9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(str(anno) + '-06-01', str(anno) + '-09-30')
            L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(str(anno) + '-06-01', str(anno) + '-09-30')
            bands = ['ST_B10', 'QA_PIXEL']
            bandName = ['ST', 'QA_PIXEL']
            filtered_L9 = L9.filter(ee.Filter.lt('CLOUD_COVER', 40)).select(bands, bandName)#.map(cloudMask)
            filtered_L8 = L8.filter(ee.Filter.lt('CLOUD_COVER', 40)).select(bands, bandName)#.map(cloudMask)
            LandsatColl = filtered_L8.merge(filtered_L9)
            LandsatColl = LandsatColl.select('ST').filterBounds(region)

            if LandsatColl.size().getInfo() > 0:

              lst = LandsatColl.max().clip(fc)
              thermal = lst.select('ST.*').multiply(0.00341802).add(149.0).subtract(273.15)


              # zonal stats per sezioni
              print('scarico immagine', comune, anno, "n° acquisizioni", LandsatColl.size().getInfo())

              scarica(thermal, nome_img, region)
            else:
              print('nessuna immagine nel periodo selezionato')
              continue
        else:
            print(nome_img, 'già presente')

    # se è il primo mese usa il df delle sezioni originario altrimenti quello con precedenti stats
        try:
            df_o = sdf
            #print("sdf esistente")
        except:
            df_o = df_sez11
            #stats = zonal_stats(df_o, nome_img, stats="mean")

            #print("sdf non esistente")
        stats = zonal_stats(df_o, nome_img, stats=statistiche)
        sdf = pd.DataFrame(stats)
        #sdf.rename(columns={stat_map: 'Temp_' + stat_map + str(anno)}, inplace=True)
        sdf = sdf.round(2)
        sdf = pd.concat([df_o, sdf], axis=1)
        stats.clear()

        #print('regione', reg, comune, 'anno', anno, 'mese', mese)#, center)#, aoi)

    #sdf1 = sdf.drop('geometry', axis=1) # sdf.drop('geometry', inplace=True, axis=1)



        # rinomina campi
        class_intervals = [0, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]     #, 100]
        sdf['class'] = pd.cut(sdf[stat_map], bins=class_intervals, labels=False)
        sdf[["P1","P14","P15","P28","P29"]] = sdf[["P1","P14","P15","P28","P29"]].fillna(0)
        sdf.rename(columns={'P1': 'Tot_residenti'}, inplace=True)#.astype('int')
        sdf['Bambini_0-9'] = (sdf['P14'].astype('int') + sdf['P15'].astype('int')).astype('int')
        sdf['Anziani_>70'] = (sdf['P28'].astype('int') + sdf['P29'].astype('int')).astype('int')
        sdf2 = sdf.loc[:, ['geometry'] + ["SEZ", 'min', 'mean', 'max', 'median','Tot_residenti','Bambini_0-9','Anziani_>70','class' ]].copy()


        #break
        sdf2.to_excel(comune + '_' + 'stats.xlsx', index=False, columns=['SEZ','min', 'mean', 'max', 'median',"Tot_residenti","Bambini_0-9","Anziani_>70"])
        #sdf2.to_csv(comune + '_' + 'stats.csv', index=False, columns=['SEZ','min', 'mean', 'max', 'median', "Tot_residenti","Bambini_0-9","Anziani_>70"])

        #del sdf
        print("Create statistiche", comune, "estate", anno)

        # Crea la mappa per il comune e in estate
        centro1 = sdf2.to_crs(epsg=3857).centroid.to_crs(df_sez11.crs)
        m = folium.Map(location=(centro1.y[0],centro1.x[0] ), tiles="cartodb positron", zoom_start=12)
        folium.Choropleth(
            geo_data=sdf2,
            name ="Temperature SUPERFICIALI per sezione 2011 - " + comune + '-' + 'estate'+ str(anno),
            data=sdf2,
            columns=["SEZ", stat_map,"Tot_residenti","Bambini_0-9","Anziani_>70"],
            key_on="feature.properties.SEZ",
            fill_color= 'RdYlGn_r', #'YlOrRd',
            bins=[20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],          # 100],
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name="Temperatura (" + stat_map + ") superficiale estiva per sezioni 2011",
        ).add_to(m)

        style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.1, 'weight': 0.1}
        SuburbName = folium.features.GeoJson(sdf2, style_function=style_function,
            control=False, tooltip=folium.features.GeoJsonTooltip(
                fields=['SEZ', stat_map, "Tot_residenti","Bambini_0-9","Anziani_>70"],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
            )
        )
        m.add_child(SuburbName)
        m.keep_in_front(SuburbName)
        basemaps['Google Satellite'].add_to(m)

        folium.LayerControl().add_to(m)
        m.save(comune + '_' + str(anno) + '.html')
        print("Creata mappa interattiva per", comune, "estate", anno)

        #break


--- Milano ---
ST_Milano_2023.tif già presente
Create statistiche Milano estate 2023
Creata mappa interattiva per Milano estate 2023
--- Aosta ---
scarico immagine Aosta 2023 n° acquisizioni 8
Create statistiche Aosta estate 2023
Creata mappa interattiva per Aosta estate 2023
--- Ancona ---
scarico immagine Ancona 2023 n° acquisizioni 35
Create statistiche Ancona estate 2023
Creata mappa interattiva per Ancona estate 2023
--- Potenza ---
scarico immagine Potenza 2023 n° acquisizioni 33
Create statistiche Potenza estate 2023
Creata mappa interattiva per Potenza estate 2023
--- Genova ---
scarico immagine Genova 2023 n° acquisizioni 21
Create statistiche Genova estate 2023
Creata mappa interattiva per Genova estate 2023
--- Firenze ---
scarico immagine Firenze 2023 n° acquisizioni 23
Create statistiche Firenze estate 2023
Creata mappa interattiva per Firenze estate 2023
--- Venezia ---
scarico immagine Venezia 2023 n° acquisizioni 44
Create statistiche Venezia estate 2023
Creata mappa in

In [ ]:
# scaricare i file html e xlsx sul pc locale
from google.colab import files
import glob

html_list = glob.glob('*.html')
xlsx_list = glob.glob('*.xlsx')

for file_h in html_list:
  files.download(file_h)

for file_x in xlsx_list:
  files.download(file_x)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>